In [2]:
import cv2 as cv
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
from pathlib import Path
from tqdm import tqdm

In [3]:
import sys
sys.path.insert(0, '../../')
from utils.general import  *

In [38]:
def shrink_image(img_path, label_path, kernel_size=2, avg_bbox_size=16272):
    bbox_sizes = []
    img = cv.imread(img_path)
    labels = pd.DataFrame({'class':[],'segment':[]})
    kernel = np.ones((kernel_size, kernel_size), dtype=float) / kernel_size**2

    with open(label_path) as f:
        for line in f:
            split_line = line.split(' ')
            cl = int(split_line[0])
            segment = np.array(split_line[1:], dtype=float)
            row = pd.DataFrame({'class':cl, 'segment': [segment]})
            labels = pd.concat([labels, row], ignore_index=True)
    
    while True: 
        bbox_sizes = []
        mean_img = cv.filter2D(img, -1, kernel)
        smaller_img = mean_img[::kernel_size, ::kernel_size]
        ysize = smaller_img.shape[0]
        xsize = smaller_img.shape[1]
        for idx, row in labels.iterrows():
            segment = row['segment']
            segment = segment.reshape(-1, 2)
            bbox = segment2box(segment)
            x, y, w, h = bbox
            w*=xsize
            h*=ysize
            bbox_sizes.append(w*h)
        bbox_mean = np.mean(bbox_sizes)
        if bbox_mean < avg_bbox_size:
            img = smaller_img
            break
        img = smaller_img

    return img, labels

In [40]:
def create_small_train_imgs(train_dir_path, 
                            label_dir_path, 
                            num_imgs=None, 
                            chosen_images = None,
                            avg_bbox_size=16272):

    assert not ((num_imgs is not None) and (chosen_images is not None)), "Must provide either num_imgs, or chosen_images"

    created_images = []
    select_random = False

    if not chosen_images:
        chosen_images = []
        select_random = True
    else:
        num_imgs = len(chosen_images)
        
    train_dir_path = Path(train_dir_path)
    label_dir_path = Path(label_dir_path)
    img_dir_list = os.listdir(train_dir_path)

    for i in tqdm(range(num_imgs)):

        if select_random:
            img_path = train_dir_path / Path(np.random.choice(img_dir_list, replace=False))
        else:
            img_path = train_dir_path / chosen_images[i]
        chosen_images.append(img_path.name)
        label_path = label_dir_path / img_path.name.replace('.jpg','.txt')

        if label_path.exists():
            small_img, small_labels = shrink_image(img_path, label_path)

            new_file_name = img_path.name.replace('.jpg', 'sm')
            small_img_train_path = train_dir_path / (new_file_name + '.jpg')
            small_img_label_path = label_dir_path / (new_file_name + '.txt')

            created_images.append(small_img_train_path)
            cv2.imwrite(small_img_train_path, small_img)
            shutil.copy(label_path, small_img_label_path)
    
    train_txt_path = train_dir_path.parent.parent / 'train2017.txt'

    with open(train_txt_path, 'a') as f:
        new_train_path = Path(new_train_dir)
        new_train_list = os.listdir(new_train_path)
        for filepath in created_images:
            img_path = Path(filepath)
            f.write('\n./images/train2017/' + img_path.name)
    
    return chosen_images

train_img_dir = '/home/isaacp/research/repos/yolov7/coco_mean_pooling/images/train2017'
train_label_dir = '/home/isaacp/research/repos/yolov7/coco_mean_pooling/labels/train2017'
num_images = 30000

chosen_images = create_small_train_imgs(train_img_dir, train_label_dir, num_images)

train_img_dir = '/home/isaacp/research/repos/yolov7/coco_extra_training/images/train2017'
train_label_dir = '/home/isaacp/research/repos/yolov7/coco_extra_training/labels/train2017'
create_small_train_imgs(train_img_dir, train_label_dir, chosen_images=chosen_images, avg_bbox_size=np.inf)

  6%|▌         | 1666/30000 [00:20<05:38, 83.76it/s]